In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -U spacy
import spacy
nlp = spacy.load("en_core_web_sm")

import random
import re
import math
import pickle
from itertools import product
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os
%cd '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/'
!pip install imohash

from muss.utils.helpers import *


train_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/train_df_0911")
valid_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/valid_df_0911")
test_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/test_df_0911")

def list_duplicates_of(seq,item):
  start_at = -1
  locs = []
  while True:
      try:
          loc = seq.index(item,start_at+1)
      except ValueError:
          break
      else:
          locs.append(loc)
          start_at = loc
  return locs

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/Baseline_MUSS_corrupt/muss
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
train_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/train_df_0911")
valid_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/valid_df_0911")
test_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/test_df_0911")

In [3]:
train_df

,level_0,index,complex,simple,complex_NER,simple_NER,co_occur_NER
0,0,0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, the period 1809, Anna, 1...","[James Edward, Anna Austen, 1814]","[James Edward, Austen, Anna, 1814]"
1,1,1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal..."
2,2,2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, six or three, Persephone,...","[Underworld, Persephone, Demeter, Hades]","[Persephone, underworld, Hades]"
3,3,3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration]
4,4,4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[city]
...,...,...,...,...,...,...,...
249161,254583,296397,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok]
249162,254584,296398,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[ne in the Gironde, Saint-Maixant, Aquitaine i..."
249163,254585,296399,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[walk]
249164,254586,296400,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe]","[Boulogne-Billancourt, Europe]"


In [4]:
# train_df = train_df.loc[train_df['co_occur_NER'].astype(str) !='[]']
# valid_df = valid_df.loc[valid_df['co_occur_NER'].astype(str) !='[]']
# test_df = test_df.loc[test_df['co_occur_NER'].astype(str) !='[]']



In [5]:
# test_simple = [sen for sen in test_df.simple]
# train_simple = [sen for sen in train_df.simple]
# valid_simple = [sen for sen in valid_df.simple]

In [6]:
### Encode the complex sentence 


def encode_complex(sentence,NER):
  def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs
  NER = list(set(NER))
  tag_sen = sentence
  tag = ' <POS_>'
  for ner in NER:
    if ner == '':
      pass
    else:
      
      index = list_duplicates_of(tag_sen,ner)
      # print(index)
      for i, idx in enumerate(index):
        # print(idx)
        # print(tag_sen)
        # print(ner)
        tag_sen = tag_sen[0:idx] + '<POS_> ' + ner + ' <_POS>' + tag_sen[idx+len(ner):]
        # print(tag_sen)
        # print('-'*30)
        try:
          index[i+1] += 2*len(tag)
        except:
          pass
      
  # tag_sen = ' <NEXT_NE> '+' <NEXT_NE> '.join(NER) + " <SENT_START> " + tag_sen
  return tag_sen


In [7]:
train_df.complex[0]
train_df.co_occur_NER[0]

['James Edward', 'Austen', 'Anna', '1814']

In [8]:
# encode_complex(train_df.complex[21],train_df.co_occur_NER[21])
train_df.complex[19:21]
# len(train_complex)

19    A very wide covered footbridge joins all platf...
20    Its widespread presence in many seeds and frui...
Name: complex, dtype: object

In [9]:
train_complex = []
train_simple = []
sentence = train_df.complex.tolist()
sentence_simp = train_df.simple.tolist()
NERs = train_df.co_occur_NER.tolist()
counter = 0
for i,j in enumerate(sentence):
  # if counter > 100:
  #   break
  try:
    if NERs[i] != ['']:
      train_complex.append(encode_complex(j,NERs[i]))
      train_simple.append(sentence_simp[i])
    # print(j,'\n',train_df.co_occur_NER[i])
    # print(encode_complex(j,train_df.co_occur_NER[i]))
    # print('-'*30)
      # print(NERs[i])
    # print('-'*50)
  except:
    pass
  # counter+=1


In [10]:
valid_complex = []
valid_simple = []
sentence = valid_df.complex.tolist()
sentence_simp = valid_df.simple.tolist()
NERs = valid_df.co_occur_NER.tolist()
counter = 0
for i,j in enumerate(sentence):
  # if counter > 100:
  #   break
  try:
    if NERs[i] != ['']:
      valid_complex.append(encode_complex(j,NERs[i]))
      valid_simple.append(sentence_simp[i])
    # print(j,'\n',valid_df.co_occur_NER[i])
    # print(encode_complex(j,valid_df.co_occur_NER[i]))

      # print(NERs[i])
    print('-'*50)
  except:
    pass
  # counter+=1

--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
-------------------------------

In [11]:
test_complex = []
test_simple = []
sentence = test_df.complex.tolist()
sentence_simp = test_df.simple.tolist()
NERs = test_df.co_occur_NER.tolist()
counter = 0
for i,j in enumerate(sentence):
  # if counter > 100:
  #   break
  try:
    if NERs[i] != ['']:
      test_complex.append(encode_complex(j,NERs[i]))
      test_simple.append(sentence_simp[i])
    # print(j,test_df.co_occur_NER[i])
    # print(encode_complex(j,test_df.co_occur_NER[i]))

    # print(NERs[i])
    # print('-'*50)
  except:
    pass
  # counter+=1

In [12]:
data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/wikilarge_tag/'

PHASES = ['train','valid', 'test']
LANGUAGES = ['complex','simple']

# train_complex[0:100]

# count = 0
for phase, language in product(PHASES, LANGUAGES):
  # print(train_input[:10])
  print(phase+'.'+language)
  data_name = phase+'_'+language
  with open(data_dir+phase+'.'+language,'w') as f:
    for sen in globals()[data_name]:
      if type(sen) == str:
        f.write("{}\n".format(sen))
  print(len(read_lines(data_dir+phase+'.'+language)))
  
  # count += 1
f.close()

# read_lines(data_dir+phase+'.'+LANGUAGES[0])

train.complex
229914
train.simple
229914
valid.complex
745
valid.simple
745
test.complex
321
test.simple
321


In [13]:
read_lines(data_dir+PHASES[1]+'.'+LANGUAGES[0])

['<POS_> Upper Sorbian <_POS> is a m<POS_> inority <_POS> language spoken by Sorbs <POS_> in Germany <_POS> in the historical province of Upper Lusatia (Hornja Å uÅ 3\\/4 ica in Sorbian), which is today part of <POS_> Saxony <_POS>.',
 'It used to be on display as part of the <POS_> American Treasures <_POS> exhibition of the <POS_> Library of Congress in Washington <_POS>, <POS_> D.C. <_POS>',
 'Relation to the Nobel Prize <POS_> The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel <_POS> is not a Nobel Prize.',
 'Of <POS_> the e <_POS>leven tropical cyclones that formed during <POS_> the season <_POS>, four affected land, with the most notable storm of <POS_> the season <_POS> being Hurricane Ismael, which killed at least 116 people in Mexico.',
 'In <POS_> 2007 <_POS>/<POS_> 08 <_POS> it had <POS_> over 40,000 <_POS> students studying <POS_> 500 <_POS> academic programmes and <POS_> more than 10,000 <_POS> staff, making it the largest single-site university in 

In [14]:
len(read_lines(data_dir+PHASES[1]+'.'+LANGUAGES[1]))

745

# Encode wikilarge prefix + tag

In [15]:
train_complex = []
train_simple = []
sentence = train_df.complex.tolist()
sentence_simp = train_df.simple.tolist()
NERs = train_df.co_occur_NER.tolist()
print(NERs[:10])
counter = 0
for i,j in enumerate(sentence):
  try:
    if NERs[i] != ['']:
      prefix = ' <NEXT_NE> '+' <NEXT_NE> '.join(NERs[i]) + " <SENT_START> " 
  
      train_complex.append(prefix + encode_complex(j,NERs[i]))
      train_simple.append(sentence_simp[i])
    # print(j,'\n',train_df.co_occur_NER[i])
    # print(encode_complex(j,train_df.co_occur_NER[i]))
    # print('-'*30)
      # print(NERs[i])
    # print('-'*50)
  except:
    pass
train_complex[:10]

[['James Edward', 'Austen', 'Anna', '1814'], ['Psalms of Thomas', 'Säve-Söderberg', "Mani's Psalms", 'Mandaean'], ['Persephone', 'underworld', 'Hades'], ['Cogeneration'], ['city'], ['ten years later', 'Suzuka', 'Japan', 'F1'], ['its first month', 'August 1909'], ['Hawaiian Independence Day', "The Kingdom of Hawai'i", 'the United Kingdom', "Ka La Ku'oko '", 'France'], ['Oxford English Dictionary', 'scythe', '1933'], ['Toshihide Saito', 'Japanese']]


[' <NEXT_NE> James Edward <NEXT_NE> Austen <NEXT_NE> Anna <NEXT_NE> 1814 <SENT_START> There is manuscript evidence that <POS_> Austen <_POS> continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, <POS_> Anna <_POS> and <POS_> James Edward <_POS> <POS_> Austen <_POS>, made further additions as late as <POS_> 1814 <_POS>.',
 " <NEXT_NE> Psalms of Thomas <NEXT_NE> Säve-Söderberg <NEXT_NE> Mani's Psalms <NEXT_NE> Mandaean <SENT_START> In a remarkable comparative analysis, <POS_> Mandaean <_POS> scholar <POS_> Säve-Söderberg <_POS> demonstrated that Mani's <POS_> Psalms of Thomas <_POS> were closely related to <POS_> Mandaean <_POS> texts.",
 ' <NEXT_NE> Persephone <NEXT_NE> underworld <NEXT_NE> Hades <SENT_START> Before <POS_> Persephone <_POS> was released to Hermes, who had been sent to retrieve her, <POS_> Hades <_POS> tricked her into eating pomegranate seeds, (six or three according to the telling) which forced her to return to the <POS_> un

In [16]:
valid_complex = []
valid_simple = []
sentence = valid_df.complex.tolist()
sentence_simp = valid_df.simple.tolist()
NERs = valid_df.co_occur_NER.tolist()
print(NERs[:10])
counter = 0
for i,j in enumerate(sentence):
  try:
    if NERs[i] != ['']:
      prefix = ' <NEXT_NE> '+' <NEXT_NE> '.join(NERs[i]) + " <SENT_START> " 
  
      valid_complex.append(prefix + encode_complex(j,NERs[i]))
      valid_simple.append(sentence_simp[i])
    # print(j,'\n',valid_df.co_occur_NER[i])
    # print(encode_complex(j,valid_df.co_occur_NER[i]))
    # print('-'*30)
      # print(NERs[i])
    # print('-'*50)
  except:
    pass
valid_complex[:10]

[['Upper Sorbian', 'in Germany', 'inority', 'Saxony'], ['Library of Congress in Washington', 'American Treasures', 'D.C.'], ['The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel'], [''], ['the season', 'the e'], ['more than 10,000', 'over 40,000', '2007', '500', '08'], ['0804', '9266', 'N35', '73'], [], ['Aarberg'], ['hippie counterculture', 'The Summer of Love']]


[' <NEXT_NE> Upper Sorbian <NEXT_NE> in Germany <NEXT_NE> inority <NEXT_NE> Saxony <SENT_START> <POS_> Upper Sorbian <_POS> is a m<POS_> inority <_POS> language spoken by Sorbs <POS_> in Germany <_POS> in the historical province of Upper Lusatia (Hornja Å uÅ 3\\/4 ica in Sorbian), which is today part of <POS_> Saxony <_POS>.',
 ' <NEXT_NE> Library of Congress in Washington <NEXT_NE> American Treasures <NEXT_NE> D.C. <SENT_START> It used to be on display as part of the <POS_> American Treasures <_POS> exhibition of the <POS_> Library of Congress in Washington <_POS>, <POS_> D.C. <_POS>',
 ' <NEXT_NE> The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel <SENT_START> Relation to the Nobel Prize <POS_> The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel <_POS> is not a Nobel Prize.',
 ' <NEXT_NE> the season <NEXT_NE> the e <SENT_START> Of <POS_> the e <_POS>leven tropical cyclones that formed during <POS_> the season <_POS>, four affected land, 

In [17]:
test_complex = []
test_simple = []
sentence = test_df.complex.tolist()
sentence_simp = test_df.simple.tolist()
NERs = test_df.co_occur_NER.tolist()
print(NERs[:10])
counter = 0
for i,j in enumerate(sentence):
  try:
    if NERs[i] != ['']:
      prefix = ' <NEXT_NE> '+' <NEXT_NE> '.join(NERs[i]) + " <SENT_START> " 
  
      test_complex.append(prefix + encode_complex(j,NERs[i]))
      test_simple.append(sentence_simp[i])
    # print(j,'\n',test_df.co_occur_NER[i])
    # print(encode_complex(j,test_df.co_occur_NER[i]))
    # print('-'*30)
      # print(NERs[i])
    # print('-'*50)
  except:
    pass
test_complex[:10]

[['Afro-Arab Abbala', 'Janjaweed', 'Sudanese', 'Rizeigat'], ['Muslims', 'Jeddah', 'Mecca', 'Islam'], ['Great Dark Spot', 'Neptune'], ['Saturday'], ['east'], ['six weeks later', 'January', '888', '13'], ['Papua New Guinea'], ['Colin Mears Award', 'the Kate Green', 'Medal', '2000', '5000'], [''], ['Giovanni Domenico Cassini', 'Christiaan Huygens', 'Italian-French', 'NASA Cassini', 'ESA Huygens', 'Dutch']]


[' <NEXT_NE> Afro-Arab Abbala <NEXT_NE> Janjaweed <NEXT_NE> Sudanese <NEXT_NE> Rizeigat <SENT_START> One side of the armed conflicts is composed mainly of the <POS_> Sudanese <_POS> military and the <POS_> Janjaweed <_POS>, a <POS_> Sudanese <_POS> militia group recruited mostly from the <POS_> Afro-Arab Abbala <_POS> tribes of the northern <POS_> Rizeigat <_POS> region in Sudan.',
 " <NEXT_NE> Muslims <NEXT_NE> Jeddah <NEXT_NE> Mecca <NEXT_NE> Islam <SENT_START> <POS_> Jeddah <_POS> is the principal gateway to <POS_> Mecca <_POS>, <POS_> Islam <_POS>'s holiest city, which able-bodied <POS_> Muslims <_POS> are required to visit at least once in their lifetime.",
 ' <NEXT_NE> Great Dark Spot <NEXT_NE> Neptune <SENT_START> The <POS_> Great Dark Spot <_POS> is thought to represent a hole in the methane cloud deck of <POS_> Neptune <_POS>.',
 ' <NEXT_NE> Saturday <SENT_START> His next work, <POS_> Saturday <_POS>, follows an especially eventful day in the life of a successful neurosurgeon.

In [18]:
data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/wikilarge_prefix_tag/'

PHASES = ['train','valid', 'test']
LANGUAGES = ['complex','simple']

# train_complex[0:100]

# count = 0
for phase, language in product(PHASES, LANGUAGES):
  # print(train_input[:10])
  print(phase+'.'+language)
  data_name = phase+'_'+language
  with open(data_dir+phase+'.'+language,'w') as f:
    for sen in globals()[data_name]:
      if type(sen) == str:
        f.write("{}\n".format(sen))
  print(len(read_lines(data_dir+phase+'.'+language)))
  
  # count += 1
f.close()

train.complex
229914
train.simple
229914
valid.complex
745
valid.simple
745
test.complex
321
test.simple
321


#Encode Asset - only tags


In [7]:
asset_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_ne2/'

asset_valid = asset_dir + 'valid.complex'
asset_test = asset_dir + 'test.complex'

read_lines(asset_test)[:10]

[' <NEXT_NE> side <NEXT_NE> Janjaweed, <NEXT_NE> Abbala <NEXT_NE> Rizeigat <SENT_START> One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.',
 " <NEXT_NE> Jeddah <NEXT_NE> Islam's <NEXT_NE> holiest <SENT_START> Jeddah is the principal gateway to Mecca, Islam's holiest city, which able-bodied Muslims are required to visit at least once in their lifetime.",
 ' <NEXT_NE> Great <NEXT_NE> Dark <NEXT_NE> Spot <NEXT_NE> thought <NEXT_NE> hole <SENT_START> The Great Dark Spot is thought to represent a hole in the methane cloud deck of Neptune.',
 ' <NEXT_NE> eventful <NEXT_NE> day <NEXT_NE> neurosurgeon. <SENT_START> His next work, Saturday, follows an especially eventful day in the life of a successful neurosurgeon.',
 ' <NEXT_NE> tarantula, <NEXT_NE> trickster <NEXT_NE> character, <NEXT_NE> spun <NEXT_NE> and, <NEXT_NE> attaching <NEX

In [25]:
asset_valid_tag = []
# train_simple = []
# sentence = train_df.complex.tolist()
# sentence_simp = train_df.simple.tolist()
NERs = [NE_extraction(i)[0]['NE_simple'] for i in read_lines(asset_valid)]
print(NERs)
counter = 0
for i,j in enumerate(read_lines(asset_valid)):
  # if counter > 100:
  #   break
  try:
    if NERs[i] != ['']:
      asset_valid_tag.append(encode_complex(NE_extraction(j)[1],NERs[i]))
      # train_simple.append(sentence_simp[i])
    # print(j,'\n')
    # print(encode_complex(j,train_df.co_occur_NER[i]))
    # print('-'*30)
      # print(NERs[i])
    # print('-'*50)
    else:
      asset_valid_tag.append(NE_extraction(j)[1])
  except:
    asset_valid_tag.append(NE_extraction(j)[1])
  # counter+=1

print(asset_valid_tag[:10])
print(len(asset_valid_tag))







[[''], ['inscription', 'attests'], [''], ["Jameson's", 'autobiography,', 'Porn', 'Cautionary'], ['particularly', 'kiwifruit.'], ['Singles', '–', 'tag', 'team', 'division,', 'took'], ['town-wide', 'projects', 'Stevenage".'], ['Word', 'processing', 'ability', 'configuration', 'time'], ['Rollins'], ['History', 'Landsberg', 'prison,'], ['hunting', 'dog', 'refers', 'dog', 'assists'], ['Nicaea', 'capturing', 'Constantinople'], [''], ['Marcel', 'Junod', '-'], ["Whisenhunt's", 'career', '-', 'Falcons,', 'twelfth', 'round,', 'then', 'spells'], ['ceded', 'Bessarabia', 'Dobruja.'], [''], [''], ['visits', 'reported', 'flag'], [''], ['goaltender'], ['extratropical'], ['Tracking', 'northwestward,', 'Fort', 'Beach', 'dissipated'], ['Anthony', 'clarinetist'], [''], [''], ['2.79', 'rating,', 'show'], ['claims', 'impaled'], ['name', 'making', 'nests'], [''], [''], ['Hager', 'reassigned', 'Wrestling,', 'TJ'], ['rock', 'characteristically'], ['absent', 'city', 'fortifications'], ['Bay'], ['Adrastea', 'Jew

In [26]:

# encode_complex(NE_extraction(read_lines(asset_valid)[89])[1],NERs[89])


In [27]:
asset_test_tag = []
# train_simple = []
# sentence = train_df.complex.tolist()
# sentence_simp = train_df.simple.tolist()
NERs = [NE_extraction(i)[0]['NE_simple'] for i in read_lines(asset_test)]
print(NERs)
counter = 0
for i,j in enumerate(read_lines(asset_test)):
  # if counter > 100:
  #   break
  try:
    if NERs[i] != ['']:
      asset_test_tag.append(encode_complex(NE_extraction(j)[1],NERs[i]))
      # train_simple.append(sentence_simp[i])
    # print(j,'\n')
    # print(encode_complex(j,train_df.co_occur_NER[i]))
    # print('-'*30)
      # print(NERs[i])
    # print('-'*50)
    else:
      asset_test_tag.append(NE_extraction(j)[1])
  except:
    asset_test_tag.append(NE_extraction(j)[1])
  # counter+=1

print(asset_test_tag[:10])
print(len(asset_test_tag))



[['side', 'Janjaweed,', 'Abbala', 'Rizeigat'], ['Jeddah', "Islam's", 'holiest'], ['Great', 'Dark', 'Spot', 'thought', 'hole'], ['eventful', 'day', 'neurosurgeon.'], ['tarantula,', 'trickster', 'character,', 'spun', 'and,', 'attaching', 'ball,', 'away', 'fast'], ['died'], ['culturally', 'akin'], ['Greenaway', 'Colin', 'Mears'], [''], ['Cassini', 'orbiter,', 'Domenico', 'Cassini,', 'Huygens', 'Christiaan', 'Huygens.'], ['("Sandro")', 'Mazzola'], ['was', 'craters.'], ['Wheaton'], ['However,', 'BZÖ', 'differs', 'bit', 'Party,', 'Lisbon'], ['species', 'had', 'vanished'], ['Wexler'], ['dextromethorphan', 'occurs'], ['Tsinghua'], ['NRC'], ['coast', 'encloses', 'city', 'Stralsund.'], ['was', 'also'], ['Fives', 'racquet'], ['King', 'Bhumibol', 'Monday,', 'so'], [''], ['Nevertheless,', 'Tagore', 'emulated', 'craftwork', 'Haida', 'carvings', 'woodcuts', 'Pechstein.'], ['candidate'], ['Great', 'Performances'], [''], ['remained', 'husband'], ['Despina'], ['Prix', 'motor', 'took', 'place'], ['also',

In [28]:
data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_tag/'

PHASES = ['valid', 'test']
LANGUAGES = ['complex']

# train_complex[0:100]
data_name = ['asset_valid_tag','asset_test_tag']
count = 0
for phase, language in product(PHASES, LANGUAGES):
  # print(train_input[:10])
  print(phase+'.'+language)
  # data_name = phase+'_'+language
  with open(data_dir+phase+'.'+language,'w') as f:
    for sen in globals()[data_name[count]]:
      if type(sen) == str:
        f.write("{}\n".format(sen))
  print(len(read_lines(data_dir+phase+'.'+language)))
  
  count += 1
f.close()

valid.complex
2000
test.complex
359


# Encode ASSET prefix + Tag

In [ ]:
asset_valid_tag = []
# train_simple = []
# sentence = train_df.complex.tolist()
# sentence_simp = train_df.simple.tolist()
NERs = [NE_extraction(i)[0]['NE_simple'] for i in read_lines(asset_valid)]
# print(NERs)
counter = 0
for i,j in enumerate(read_lines(asset_valid)):
  # if counter > 100:
  #   break
  try:
    if NERs[i] != ['']:
      prefix = j[:list_duplicates_of(j,'<SENT_START>')[0]+len(' <SENT_START>')]
      # print(list_duplicates_of(j,'<SENT_START>'))

      asset_valid_tag.append(prefix + encode_complex(NE_extraction(j)[1],NERs[i]))
      # train_simple.append(sentence_simp[i])
    # print(j,'\n')
    # print(encode_complex(j,train_df.co_occur_NER[i]))
    # print('-'*30)
      # print(NERs[i])
    # print('-'*50)
    else:
      asset_valid_tag.append(NE_extraction(j)[1])
  except:
    print('error')
    asset_valid_tag.append(NE_extraction(j)[1])
  # counter+=1

print(asset_valid_tag[:10])
print(len(asset_valid_tag))



['Adjacent counties are Marin (to the south), Mendocino (to the north), Lake (northeast), Napa (to the east), and Solano and Contra Costa (to the southeast).', ' <NEXT_NE> inscription <NEXT_NE> attests <SENT_START> A Georgian <POS_> inscription <_POS> around the drum <POS_> attests <_POS> his name.', 'They would later return to the revived series in the 2008 Christmas Special "The Next Doctor", introducing two new variants of the race; the Cyber-Shades and the Cyber-King.', " <NEXT_NE> Jameson's <NEXT_NE> autobiography, <NEXT_NE> Porn <NEXT_NE> Cautionary <SENT_START> <POS_> Jameson's <_POS> <POS_> autobiography, <_POS> How to Make Love Like a <POS_> Porn <_POS> Star: A <POS_> Cautionary <_POS> Tale was published August 17, 2004.", ' <NEXT_NE> particularly <NEXT_NE> kiwifruit. <SENT_START> It is <POS_> particularly <_POS> famous for the cultivation of <POS_> kiwifruit. <_POS>', ' <NEXT_NE> Singles <NEXT_NE> – <NEXT_NE> tag <NEXT_NE> team <NEXT_NE> division, <NEXT_NE> took <SENT_START> 

In [ ]:
asset_valid_tag[:10]

['Adjacent counties are Marin (to the south), Mendocino (to the north), Lake (northeast), Napa (to the east), and Solano and Contra Costa (to the southeast).',
 ' <NEXT_NE> inscription <NEXT_NE> attests <SENT_START> A Georgian <POS_> inscription <_POS> around the drum <POS_> attests <_POS> his name.',
 'They would later return to the revived series in the 2008 Christmas Special "The Next Doctor", introducing two new variants of the race; the Cyber-Shades and the Cyber-King.',
 " <NEXT_NE> Jameson's <NEXT_NE> autobiography, <NEXT_NE> Porn <NEXT_NE> Cautionary <SENT_START> <POS_> Jameson's <_POS> <POS_> autobiography, <_POS> How to Make Love Like a <POS_> Porn <_POS> Star: A <POS_> Cautionary <_POS> Tale was published August 17, 2004.",
 ' <NEXT_NE> particularly <NEXT_NE> kiwifruit. <SENT_START> It is <POS_> particularly <_POS> famous for the cultivation of <POS_> kiwifruit. <_POS>',
 ' <NEXT_NE> Singles <NEXT_NE> – <NEXT_NE> tag <NEXT_NE> team <NEXT_NE> division, <NEXT_NE> took <SENT_ST

In [ ]:
asset_test_tag = []
# train_simple = []
# sentence = train_df.complex.tolist()
# sentence_simp = train_df.simple.tolist()
NERs = [NE_extraction(i)[0]['NE_simple'] for i in read_lines(asset_test)]
# print(NERs)
counter = 0
for i,j in enumerate(read_lines(asset_test)):
  # if counter > 100:
  #   break
  try:
    if NERs[i] != ['']:
      prefix = j[:list_duplicates_of(j,'<SENT_START>')[0]+len(' <SENT_START>')]
      # print(list_duplicates_of(j,'<SENT_START>'))

      asset_test_tag.append(prefix + encode_complex(NE_extraction(j)[1],NERs[i]))
      # train_simple.append(sentence_simp[i])
    # print(j,'\n')
    # print(encode_complex(j,train_df.co_occur_NER[i]))
    # print('-'*30)
      # print(NERs[i])
    # print('-'*50)
    else:
      asset_test_tag.append(NE_extraction(j)[1])
  except:
    print('error')
    asset_test_tag.append(NE_extraction(j)[1])
  # counter+=1

print(asset_test_tag[:10])
print(len(asset_test_tag))

[' <NEXT_NE> side <NEXT_NE> Janjaweed, <NEXT_NE> Abbala <NEXT_NE> Rizeigat <SENT_START> One <POS_> side <_POS> of the armed conflicts is composed mainly of the Sudanese military and the <POS_> Janjaweed, <_POS> a Sudanese militia group recruited mostly from the Afro-Arab <POS_> Abbala <_POS> tribes of the northern <POS_> Rizeigat <_POS> region in Sudan.', " <NEXT_NE> Jeddah <NEXT_NE> Islam's <NEXT_NE> holiest <SENT_START> <POS_> Jeddah <_POS> is the principal gateway to Mecca, <POS_> Islam's <_POS> <POS_> holiest <_POS> city, which able-bodied Muslims are required to visit at least once in their lifetime.", ' <NEXT_NE> Great <NEXT_NE> Dark <NEXT_NE> Spot <NEXT_NE> thought <NEXT_NE> hole <SENT_START> The <POS_> Great <_POS> <POS_> Dark <_POS> <POS_> Spot <_POS> is <POS_> thought <_POS> to represent a <POS_> hole <_POS> in the methane cloud deck of Neptune.', ' <NEXT_NE> eventful <NEXT_NE> day <NEXT_NE> neurosurgeon. <SENT_START> His next work, Satur<POS_> day <_POS>, follows an especial

In [ ]:
data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_prefix_tag/'

PHASES = ['valid', 'test']
LANGUAGES = ['complex']

# train_complex[0:100]
data_name = ['asset_valid_tag','asset_test_tag']
count = 0
for phase, language in product(PHASES, LANGUAGES):
  # print(train_input[:10])
  print(phase+'.'+language)
  # data_name = phase+'_'+language
  with open(data_dir+phase+'.'+language,'w') as f:
    for sen in globals()[data_name[count]]:
      if type(sen) == str:
        f.write("{}\n".format(sen))
  print(len(read_lines(data_dir+phase+'.'+language)))
  
  count += 1
f.close()

valid.complex
2000
test.complex
359
